In [8]:
from utils import *
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
df = pd.read_csv("M.csv") \
                .pipe(clean_data) 

In [10]:
df_despacho = df.pipe(return_despacho_movements)
df_hogar = df.pipe(return_hogar_movements)

In [11]:
#write_raw_to_excel(df_despacho, "Raw Despacho.xlsx")
#write_raw_to_excel(df_hogar, "Raw Hogar.xlsx")

In [12]:
df_despacho.pipe(pivot_by_category_totals, ["Tipo"])

,Tipo,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Media,Total
0,Gastos,-627.95,-655.18,-638.43,-1782.86,-593.03,-431.15,-1430.33,-283.38,-440.76,-2310.61,-490.46,-263.54,-880.38,-9947.68
1,Ingresos,410.00,3290.60,5334.69,4578.71,3413.30,3670.00,3903.71,50.00,242.00,8752.27,4446.89,1938.61,3462.92,40030.78
2,Total,-217.95,2635.42,4696.26,2795.85,2820.27,3238.85,2473.38,-233.38,-198.76,6441.66,3956.43,1675.07,2582.54,30083.10


In [13]:
df_hogar.pipe(pivot_by_category_totals, ["Tipo"])

,Tipo,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Media,Total
0,Gastos,-4858.84,-4370.48,-5208.63,-5062.51,-5210.00,-8112.99,-8256.59,-3461.10,-5234.44,-12485.53,-8507.20,-8541.50,-6433.48,-79309.81
1,Ingresos,2642.34,9696.62,7735.52,5602.14,5627.59,9110.07,5291.84,2587.06,2727.39,9266.42,10296.45,11082.86,6416.68,81666.30
2,Total,-2216.50,5326.14,2526.89,539.63,417.59,997.08,-2964.75,-874.04,-2507.05,-3219.11,1789.25,2541.36,-16.80,2356.49


In [14]:
write_to_excel(df_hogar, 'Hogar.xlsx')
write_to_excel(df_despacho, 'Despacho.xlsx')